In [52]:
import gradio as gr
import mediapipe as mp
import numpy as np
import cv2
import random
import h5py
import tensorflow as tf
from keras.models import load_model

weights_file = h5py.File('weights/smnist.h5', 'r')
model = load_model(weights_file)

phraseQuestions = [
    "What is the sign for 'Hello'?",
    "Show the sign for 'Thank You'",
    "Can you sign 'Happy'?",
    "What is the sign for 'Sorry'?",
    "Please sign 'Goodbye'"
]

letterQuestions = [
    "What is the sign for 'A'?",
    #"Show the sign for 'B'",
   #"Can you sign 'C'?",
    #"What is the sign for 'D'?",
    #"Please sign 'E'"
]

def segment(input_image, textbox):
    mp_selfie = mp.solutions.selfie_segmentation

    image_np = np.array(input_image)
    resized_image = cv2.resize(image_np, (28, 28))

    with mp_selfie.SelfieSegmentation(model_selection=0) as segmentation_model:
        res = segmentation_model.process(resized_image)
        mask = np.stack((res.segmentation_mask,) * 3, axis=-1) > 0.5
        segmented_image = np.where(mask, resized_image, cv2.blur(resized_image, (40, 40)))

        gray_image = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY)
        image = np.expand_dims(gray_image, axis=-1)
        image = image / 255.0

        predictions = model.predict(np.expand_dims(image, axis=0))
        answer = textbox.split(' ')[-1].strip("'?")
        text_to_display = 'Correct' #if predictions[0] == answer else 'Incorrect'

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color = (0, 255, 0) if text_to_display == 'Correct' else (0, 0, 255)
        thickness = 2
        textSize = cv2.getTextSize(text_to_display, font, font_scale, thickness)[0]

        textX = (image.shape[1] - textSize[0]) // 2
        textY = (image.shape[0] + textSize[1]) // 2


        cv2.putText(segmented_image, text_to_display, (textX, textY), font, font_scale, color, thickness, cv2.LINE_AA)

        return cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB)

def update_question():
    return random.choice(letterQuestions)


options = ['A', 'B', 'C']


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Tabs() as tabs:
        with gr.TabItem("Learning Mode", id=0):
            text_options = gr.Dropdown(options, label="Signs to learn")
            with gr.Row():
                t = gr.Textbox("The symbol for Happy Birthday is: ")
                img = gr.Image()
        with gr.TabItem("Game Mode", id=1):
            with gr.Row():
                question_display = gr.Textbox(label="Current Question", value=update_question(), interactive=False)
                next_question_button = gr.Button("Next Question")
                next_question_button.click(fn=update_question, inputs=[], outputs=question_display)
            with gr.Row():
                webcam_input = gr.Image(label="Start signing", shape=(640,480), source="webcam", tool="select", type="pil")
                output_image = gr.Image(label="You signed...", shape=(640,480), type="pil")
                i = gr.Interface(fn=segment, inputs=[webcam_input, question_display], outputs=output_image, live=True)
                #i.launch()
            
weights_file.close()
demo.launch()

Running on local URL:  http://127.0.0.1:7913

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 74ms/step
pred: [1.4785131e-14 1.3676394e-09 9.8835883e-08 2.3794235e-09 3.0899569e-13
 4.3699242e-16 2.9188227e-14 4.7501298e-11 9.5941558e-14 3.6912837e-08
 2.0746448e-10 5.2551071e-11 4.7484992e-11 3.7620526e-10 1.7201469e-07
 9.9996901e-01 8.2092959e-13 6.6081377e-14 4.8536056e-13 3.0685151e-05
 1.0485618e-12 1.4622074e-15 1.0987794e-07 5.4310631e-14], [[1.4785131e-14 1.3676394e-09 9.8835883e-08 2.3794235e-09 3.0899569e-13
  4.3699242e-16 2.9188227e-14 4.7501298e-11 9.5941558e-14 3.6912837e-08
  2.0746448e-10 5.2551071e-11 4.7484992e-11 3.7620526e-10 1.7201469e-07
  9.9996901e-01 8.2092959e-13 6.6081377e-14 4.8536056e-13 3.0685151e-05
  1.0485618e-12 1.4622074e-15 1.0987794e-07 5.4310631e-14]]
